# Cycle Capture Function Code

## Instructions for Using the Cycle Capture Function in Time Series Data:

### Author: Zhiyuan Xie

### Overview:

This function is designed to efficiently capture cycle information in time series data. It's particularly useful for analyzing patterns and periodicity in datasets represented as a Series in Python.

### Steps for Usage:

#### Data Preparation:

##### Ensure your data is in a Series format. 
This format is essential for the function to process the data correctly.

##### Normalize your time series data before applying the function. 
Normalization helps in reducing biases due to scale differences and enhances the accuracy of cycle detection.

#### Setting Parameters:

##### Filtering Threshold: 
The default threshold is set to filter out 0.02% of the data points. You can adjust this threshold based on your specific requirements to either include more data or apply stricter filtering.

##### Cycle Limits: 
You have the flexibility to set the upper and lower limits for cycle capture. This feature allows you to tailor the cycle detection to specific ranges, making it more relevant for your analysis.

### Advantages of Using this Function:

#### Customization: 
The ability to adjust filtering thresholds and cycle limits provides a high degree of customization to suit various datasets and analytical needs.

#### Efficiency: 
Designed to handle Series format data efficiently, ensuring fast and accurate cycle detection.

#### Enhanced Analysis: 
By capturing cycles accurately, this function aids in better understanding the underlying patterns and trends in your time series data, leading to more informed decision-making.

### Note: 

It's important to understand your data thoroughly before applying these settings, as different datasets might require unique adjustments for optimal results.

For those utilizing this code in their projects, we highly recommend consulting the paper "Direct and Indirect Hydrogen Storage: Dynamics of Interactions Within Europe's Highly Renewable Energy System". This resource offers valuable insights that can enhance your application of the code.

Please feel free to use this code in your work. We kindly request that you cite the aforementioned paper in any publications or presentations that incorporate this code.

In [ ]:
# Cycle Capture Function

def calculate_all_cycles(series,threshold_01,threshold_02):
    df = pd.DataFrame(series)
    df.columns = ['filling_level']

    df['difference'] = df['filling_level'].diff()

    df['state'] = 'waiting'
    df.loc[df['difference'] > 0.0002, 'state'] = 'charging'
    df.loc[df['difference'] < -0.0002, 'state'] = 'discharging'

    cycles = []
    
    # New Series to track state within each cycle
    state_series = pd.Series('waiting', index=series.index)
    
    current_cycle = {'start': None, 'end': None, 'length': 0, 'net_energy_charging': 0, 'net_energy_discharging': 0, 'min_charging': 0, 'max_charging': 0, 'min_discharging': 0, 'max_discharging': 0}
    charging = False
    discharging = False
    first_state = ''

    for index, row in df.iterrows():
        state = row['state']
        
        state_series.loc[index] = state
        
        filling_level = row['filling_level']
        diff = row['difference']

        # set the first_state that can help us entry the judge condition
        if not first_state and state != 'waiting':
            first_state = state
            current_cycle['start'] = series.index.get_loc(index)     
            
        # situation 1, begin with charging
        if first_state == 'charging':
            if state == 'charging':
                
                if not charging and not discharging:
                    current_cycle['start'] = series.index.get_loc(index)
                    charging = True
                elif discharging:
                    charging = True
                    discharging = False
                if current_cycle['min_charging'] == 0:
                    current_cycle['min_charging'] = filling_level
                    current_cycle['max_charging'] = filling_level
                else:
                    current_cycle['min_charging'] = min(current_cycle['min_charging'], filling_level)
                    current_cycle['max_charging'] = max(current_cycle['max_charging'], filling_level)
                
                if (current_cycle['max_charging'] - current_cycle['min_charging'] > threshold_01) and (current_cycle['max_discharging'] - current_cycle['min_discharging'] > threshold_02):
                    current_cycle['end'] = series.index.get_loc(index)
                    current_cycle['length'] = series.index.get_loc(index) - current_cycle['start']
                    current_cycle['net_energy'] = current_cycle['net_energy_charging'] - current_cycle['net_energy_discharging']
                    cycles.append(current_cycle)
                    current_cycle = {'start': series.index.get_loc(index), 'end': None, 'length': 0, 'net_energy': 0, 'net_energy_charging': 0, 'net_energy_discharging': 0, 'min_charging': 0, 'max_charging': 0, 'min_discharging': 0, 'max_discharging': 0}
                    charging = False
                    discharging = False
                
                current_cycle['net_energy_charging'] += abs(diff) if diff > 0 else 0
            
            elif state == 'discharging' and charging:
                discharging = True
                current_cycle['net_energy_discharging'] += abs(diff) if diff < 0 else 0
                if current_cycle['min_discharging'] == 0:
                    current_cycle['min_discharging'] = filling_level
                    current_cycle['max_discharging'] = filling_level
                else:
                    current_cycle['min_discharging'] = min(current_cycle['min_discharging'], filling_level)
                    current_cycle['max_discharging'] = max(current_cycle['max_discharging'], filling_level)

        # situation 2, begin with discharging
        elif first_state == 'discharging':
            if state == 'discharging':
                
                if not charging and not discharging:
                    current_cycle['start'] = series.index.get_loc(index)
                    discharging = True
                elif charging:
                    discharging = True
                    charging = False
                if current_cycle['min_discharging'] == 0:
                    current_cycle['min_discharging'] = filling_level
                    current_cycle['max_discharging'] = filling_level
                else:
                    current_cycle['min_discharging'] = min(current_cycle['min_discharging'], filling_level)
                    current_cycle['max_discharging'] = max(current_cycle['max_discharging'], filling_level)
                
                if (current_cycle['max_charging'] - current_cycle['min_charging'] > threshold_01) and (current_cycle['max_discharging'] - current_cycle['min_discharging'] > threshold_02):
                    current_cycle['end'] = series.index.get_loc(index)
                    current_cycle['length'] = series.index.get_loc(index) - current_cycle['start']
                    current_cycle['net_energy'] = current_cycle['net_energy_charging'] - current_cycle['net_energy_discharging']
                    cycles.append(current_cycle)
                    current_cycle = {'start': series.index.get_loc(index), 'end': None, 'length': 0, 'net_energy': 0, 'net_energy_charging': 0, 'net_energy_discharging': 0, 'min_charging': 0, 'max_charging': 0, 'min_discharging': 0, 'max_discharging': 0}
                    charging = False
                    discharging = False
                
                current_cycle['net_energy_discharging'] += abs(diff) if diff < 0 else 0
            
            elif state == 'charging' and discharging:
                discharging = True
                current_cycle['net_energy_charging'] += abs(diff) if diff > 0 else 0
                if current_cycle['min_charging'] == 0:
                    current_cycle['min_charging'] = filling_level
                    current_cycle['max_charging'] = filling_level
                else:
                    current_cycle['min_charging'] = min(current_cycle['min_charging'], filling_level)
                    current_cycle['max_charging'] = max(current_cycle['max_charging'], filling_level)

    new_series = pd.Series(0, index=series.index)

    for cycle in cycles:
        start_index = cycle['start']
        end_index = cycle['end']
        length = cycle['length']
        new_series.iloc[start_index:end_index] = length

    return cycles, new_series, state_series

